# CSE 258 Assignment 1 Rating
**Ming Ki Toby Cheng**

In [1]:
import gzip
from collections import defaultdict
import scipy
import scipy.optimize
from sklearn import linear_model
import numpy
import random
import matplotlib.pyplot as plt

In [2]:
def readGz(path):
    for l in gzip.open(path, "rt"):
        yield eval(l)


def readCSV(path):
    f = gzip.open(path, "rt")
    f.readline()
    for l in f:
        yield l.strip().split(",")

In [3]:
# Defining MSE function
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [4]:
def prediction(user, item):
    if user not in userBiases and item in itemBiases:
        return alpha + itemBiases[item]
    if user in userBiases and item not in itemBiases:
        return alpha + userBiases[user]
    if user not in userBiases and item not in itemBiases:
        return alpha
    if user in userBiases and item in itemBiases:
        return alpha + userBiases[user] + itemBiases[item]

In [5]:
users = []
books = []
ratings = []

for user, book, _ in readCSV("train_Interactions.csv.gz"):
    users.append(user)
    books.append(book)
    ratings.append(_)

In [6]:
all_data = list(zip(users, books, ratings))

In [7]:
all_rating = []
userRatings = defaultdict(list)
bookRatings = defaultdict(list)


for user, book, r in all_data:
    all_rating.append(int(r))
    userRatings[user].append(int(r))
    bookRatings[book].append(int(r))


In [8]:
allAverage = sum(all_rating) / len(all_rating)

alpha = allAverage
nUsers = len(userRatings)
nItems = len(bookRatings)
users = list(userRatings.keys())
items = list(bookRatings.keys())

userBiases = defaultdict(float)
itemBiases = defaultdict(float)


In [9]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [10]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in all_data]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [11]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(all_data)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in all_data:
        u,i = d[0], d[1]
        pred = prediction(u, i)
        diff = pred - int(d[2])
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

In [12]:
labels = [int(d[2]) for d in all_data]

In [13]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 0.0000168))

MSE = 1.4744154699735492
MSE = 1.4570319169995913
MSE = 1.393533889858659
MSE = 7.895479454540844
MSE = 1.370350294778541
MSE = 1.2066366132890634
MSE = 1.2051200097212715
MSE = 1.1991489285514936
MSE = 1.176789933496203
MSE = 1.063003446196276
MSE = 1.0135355949628004
MSE = 0.9839360442345014
MSE = 0.9692487850331981
MSE = 0.958484006943688
MSE = 0.9436564302519366
MSE = 0.9353114927636458
MSE = 0.9331309898907322
MSE = 0.932368037231562
MSE = 0.9306453149999646
MSE = 0.9290699136177415
MSE = 0.9283440069087131
MSE = 0.9278096284027274
MSE = 0.9274843737623956
MSE = 0.9270731872454248
MSE = 0.9266477258480118
MSE = 0.9265189166652921
MSE = 0.9265431912685078
MSE = 0.9266365358524254
MSE = 0.926699721339586
MSE = 0.9265715822785244
MSE = 0.9274028964494504
MSE = 0.9266224848116169
MSE = 0.9263279285056032
MSE = 0.9260054672023919
MSE = 0.9261801641225601
MSE = 0.9259955473589025
MSE = 0.9264065294843348
MSE = 0.9260222882906745
MSE = 0.9260071499269964
MSE = 0.9260520695654878
MSE = 0.

(array([ 3.81227183, -0.02013682,  0.257077  , ..., -0.28247759,
        -0.48340748, -0.24948904]),
 0.997802847685844,
 {'grad': array([-7.40579398e-06, -1.31068698e-07,  1.96913843e-07, ...,
          4.54754267e-07,  5.88199412e-08,  8.77511889e-07]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 133,
  'nit': 114,
  'warnflag': 0})

In [14]:
# Writing predictions of test set to file
predictions = open("predictions_Rating_Assignment1_v3.1.txt", "w")
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    
    predictions.write(u + "-" + b + "," + str(prediction(u,b)) + "\n")

predictions.close()

#### Using previous lambda now use all the data available

In [15]:
all_ratngs = list(zip(users, books, ratings))

In [16]:
# Forming dictionary with user and book combinations
ratingPerCombo = {}
usersPerBook = defaultdict(set)
booksPerUser = defaultdict(set)

for user, book,r in all_ratings:
    usersPerBook[book].add(user)
    booksPerUser[user].add(book)
    ratingPerCombo[(user,book)] = int(r)

NameError: name 'all_ratings' is not defined

In [ ]:
# Initializing all user ratings and all books dictionaries
totalRatings = []
userRatings = defaultdict(list)
bookRatings = defaultdict(list)

# All ratings for each user and each book
for user, book, r in all_ratings:
    r = int(r)
    totalRatings.append(r)
    userRatings[user].append(r)
    bookRatings[book].append(r)

globalAverage = sum(totalRatings) / len(totalRatings)
userAverage = {}
userBias = {}
user_total = 0
bookAverage = {}
bookBias = {}
book_total = 0

# Initializing user and book biases
for user in userRatings:
    userAverage[user] = sum(userRatings[user]) / len(userRatings[user])
    
for user in userAverage:
    user_total += float(userAverage[user])
    
for user in userAverage:
    userBias[user] = userAverage[user] - (user_total/len(userAverage))

for book in bookRatings:
    bookAverage[book] = sum(bookRatings[book]) / len(bookRatings[book])
    
for book in bookAverage:
    book_total += float(bookAverage[book])
    
for book in bookAverage:
    bookBias[book] = bookAverage[book] - (book_total/len(bookAverage))

In [ ]:
# Defining alpha, bookBias, userBias by convergence from Training data and lambda value 1
lamb = 3.15
alpha_sum = 0
alpha = 0
MSE_diff = 5
trial = 0

while MSE_diff > 0.00005 or trial > 1000:
    model_predictions = []
    actual_rating = []
    alpha_sum = 0
    
    for user, book, r in all_ratings:
        alpha_sum += (r-(userBias[user] + bookBias[book]))
    alpha = alpha_sum/len(all_ratings)
    
    for user in booksPerUser:
        beta_U_Sum = 0
        for books in booksPerUser[user]:
            beta_U_Sum += ratingPerCombo[user,books] - (alpha + bookBias[books])
        userBias[user] = beta_U_Sum/(lamb+ len(booksPerUser[user]))
    
    for book in usersPerBook:
        beta_I_Sum = 0
        for users in usersPerBook[book]:
            beta_I_Sum += ratingPerCombo[users,book] - (alpha + userBias[users])
        bookBias[book] = beta_I_Sum/(lamb+ len(usersPerBook[book]))
    
    for user, book, r in all_ratings:
        model_predictions.append(int(prediction(user, book)))
        actual_rating.append(r)
    
    if trial == 0:
        MSE_0 = 0

    MSE_1 = MSE(model_predictions, actual_rating)
    
    MSE_diff = abs(MSE_1 - MSE_0)

    print('MSE_new:', MSE_1)
    
    MSE_0 = MSE_1
    trial += 1 

In [ ]:
# Predictions and MSE on validation data
model_predictions = []
for user, book, r in validation_rating:
    model_predictions.append(prediction(user, book))

MSE_valid = MSE(model_predictions, ratings_valid)
print('MSE on validation set:',MSE_valid)

In [ ]:
# Writing predictions of test set to file
predictions = open("predictions_Rating_Assignment1.txt", "w")
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    u, b = l.strip().split("-")
    
    predictions.write(u + "-" + b + "," + str(prediction(u,b)) + "\n")

predictions.close()

**Kaggle Username: tobycheng or Toby Cheng**

**Kaggle Rating MSE: 1.13707**